In [1]:
import torch

import numpy as np
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader

from ds.drivers.datasets import DriversDataset, all_data

In [2]:
seed = 1337
np.random.seed(seed)
torch.random.manual_seed(seed)

In [3]:
ds_all_train = DriversDataset(
    all_data, ds_type="train", data_field="working_data_filtered"
)
ds_all_test = DriversDataset(
    all_data, ds_type="test", data_field="working_data_filtered"
)

In [4]:
epoch_count = 100
train_batch_size = 8
test_batch_size = 4
device = "cuda"

In [5]:
dl_train = DataLoader(
    ds_all_train,
    batch_size=train_batch_size,
    shuffle=True,
    num_workers=1,
    pin_memory=False,
    drop_last=True,
)

In [6]:
dl_test = DataLoader(
    ds_all_test,
    batch_size=test_batch_size,
    shuffle=True,
    num_workers=1,
    pin_memory=False,
    drop_last=True,
)

In [7]:
class ConvX(nn.Module):
    def __init__(
        self, in_planes, out_planes, kernel=3, stride=1, padding=None
    ):
        super(ConvX, self).__init__()
        padding = kernel // 2 if padding is None else padding
        self.conv = nn.Conv1d(
            in_planes,
            out_planes,
            kernel_size=kernel,
            stride=stride,
            padding=padding,
            bias=False,
        )
        self.bn = nn.BatchNorm1d(out_planes)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        return self.relu(self.bn(self.conv(x)))

In [8]:
class FCRelu(nn.Module):
    def __init__(self, in_planes, out_planes):
        super(FCRelu, self).__init__()

        self.fc = nn.Linear(in_planes, out_planes)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        return self.relu(self.fc(x))

In [9]:
class Net8Conv(nn.Module):
    def __init__(self):
        super(Net8Conv, self).__init__()
        self.seq = torch.nn.Sequential(
            ConvX(1, 1, kernel=2, padding=0),
            ConvX(1, 1, kernel=2, padding=0),
            ConvX(1, 1, kernel=2, stride=2, padding=0),
            ConvX(1, 1, kernel=2, padding=0),
            ConvX(1, 1, kernel=2, padding=0),
            ConvX(1, 1, kernel=2, stride=2, padding=0),
            ConvX(1, 1, kernel=2, padding=0),
            nn.Conv1d(1, 1, kernel_size=2, stride=2),
        )
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x):
        return self.softmax(torch.flatten(self.seq(x), 1))

In [10]:
net = Net8Conv().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [11]:
def val(model):
    with torch.no_grad():
        sum_ = 0
        for i, data in enumerate(dl_test):
            inputs, labels = data
            inputs = (
                torch.unsqueeze(inputs, 1).to(torch.float32).to(device=device)
            )

            outputs = model(inputs).cpu()
            eq = outputs.max(1).indices == labels
            sum_ += eq.sum()
    return sum_

In [12]:
def train(model, epoch_count=50, print_step=50):
    min_loss = (np.inf, 0)
    best_rate = (0, 0)
    worst_rate = (1.0, 0)
    len_test = len(dl_test) * dl_test.batch_size
    for epoch in range(epoch_count):  # loop over the dataset multiple times
        running_loss = 0.0
        for i, data in enumerate(dl_train):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs = (
                torch.unsqueeze(inputs, 1).to(torch.float32).to(device=device)
            )

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs.cpu(), labels)

            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % print_step == print_step - 1:
                mean_loss = running_loss / print_step
                if mean_loss < min_loss[0]:
                    min_loss = (mean_loss, (epoch, i))
                print(f"[{epoch + 1}, {i + 1:5d}] loss: {mean_loss:.3f}")
                running_loss = 0.0
        count_of_correct = val(model)
        rate = count_of_correct / len_test
        if rate > best_rate[0]:
            best_rate = (rate, epoch)
        if rate < worst_rate[0]:
            worst_rate = (rate, epoch)

        print(
            f"[{epoch + 1}] rate: {rate:.3f} - {count_of_correct:3d}/{len_test}; {best_rate = }, {worst_rate = }"
        )
    print("Finished Training. Min_loss:", min_loss)
    return worst_rate, best_rate, min_loss

In [13]:
train(net, epoch_count=epoch_count)

[1,    50] loss: 0.702
[1,   100] loss: 0.698
[1,   150] loss: 0.691
[1] rate: 0.594 - 190/320; best_rate = (tensor(0.5938), 0), worst_rate = (tensor(0.5938), 0)
[2,    50] loss: 0.702
[2,   100] loss: 0.695
[2,   150] loss: 0.694
[2] rate: 0.597 - 191/320; best_rate = (tensor(0.5969), 1), worst_rate = (tensor(0.5938), 0)
[3,    50] loss: 0.692
[3,   100] loss: 0.697
[3,   150] loss: 0.698
[3] rate: 0.603 - 193/320; best_rate = (tensor(0.6031), 2), worst_rate = (tensor(0.5938), 0)
[4,    50] loss: 0.696
[4,   100] loss: 0.698
[4,   150] loss: 0.696
[4] rate: 0.616 - 197/320; best_rate = (tensor(0.6156), 3), worst_rate = (tensor(0.5938), 0)
[5,    50] loss: 0.697
[5,   100] loss: 0.693
[5,   150] loss: 0.698
[5] rate: 0.584 - 187/320; best_rate = (tensor(0.6156), 3), worst_rate = (tensor(0.5844), 4)
[6,    50] loss: 0.694
[6,   100] loss: 0.693
[6,   150] loss: 0.695
[6] rate: 0.600 - 192/320; best_rate = (tensor(0.6156), 3), worst_rate = (tensor(0.5844), 4)
[7,    50] loss: 0.695
[7,  

((tensor(0.5312), 77), (tensor(0.6562), 64), (0.6834311521053315, (97, 49)))

data_field="working_data"
best rate = 0.6792
min train loss = 0.680715000629425

data_field="working_data_filtered"
best rate = 0.531